## Imports and API Key Setup

In [202]:
import os
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langgraph.graph import Graph
import logging
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
from typing import Dict, Any

## Load Dell Laptop Product Pages

In [203]:
urls = [
    'https://www.dell.com/en-us/shop/laptops-2-in-1-pcs/inspiron-14-2-in-1-laptop/spd/inspiron-14-7435-2-in-1-laptop',
    'https://www.dell.com/en-us/shop/laptops-2-in-1-pcs/latitude-5450-laptop/spd/latitude-14-5450-laptop',
    'https://www.dell.com/en-us/shop/laptops-2-in-1-pcs/latitude-7450-laptop-or-2-in-1/spd/latitude-14-7450-laptop',
    'https://www.dell.com/en-us/shop/laptops-2-in-1-pcs/xps-14-laptop/spd/xps-14-9440-laptop'
]
loader = WebBaseLoader(urls)
documents = loader.load()
print(f'Loaded {len(documents)} documents.')

Loaded 4 documents.


## Split Documents and Create Vector Store:

In [204]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(splits, embeddings)
retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 3})

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## Build the LangGraph Agent

In [205]:
llm = ChatOpenAI(temperature=0.7)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
qa_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory, verbose=True)

def recommend_laptop(state: Dict[str, Any]) -> Dict[str, Any]:
    query = state['query']
    response = qa_chain({'question': query})
    return {'response': response['answer']}

graph = Graph()
graph.add_node('recommendation', recommend_laptop)
graph.set_entry_point('recommendation')
app = graph.compile()

## Testing Specific Laptop Configuration Requests

In [206]:
### Test Case 1: Travel Laptop with Intel® Core™ 7 150U

In [207]:
# Create the chain with logging
def log_retriever_output(query: str) -> str:
    logger.info(f"Retriever received query: {query}")
    docs = retriever.invoke(query)
    logger.info(f"Retriever returned {len(docs)} documents")
    for i, doc in enumerate(docs):
        logger.info(f"Document {i+1} content: {doc.page_content[:200]}...")
    context = "\n".join([doc.page_content for doc in docs])
    logger.info(f"Combined context length: {len(context)} characters")
    return context

def log_prompt_output(inputs: dict) -> dict:
    logger.info("Formatting prompt with context and question")
    logger.info(f"Input to prompt formatting: {inputs}")
    messages = prompt.format_messages(**inputs)
    logger.info(f"Formatted messages: {messages}")
    return messages

def log_llm_output(messages: list) -> str:
    logger.info("Sending messages to LLM")
    logger.info(f"Messages being sent to LLM: {messages}")
    response = llm.invoke(messages)
    logger.info(f"LLM response type: {type(response)}")
    logger.info(f"Full LLM response: {response.content}")
    return response.content

def process_response(response: str, query: str) -> Dict[str, Any]:
    if response and isinstance(response, str) and len(response.strip()) > 0:
        formatted_response = f"""Based on your requirements, here is my recommendation:

{response}

Key Features:
- Processor: {query.split('that has')[1].split('and')[0].strip() if 'that has' in query else 'Not specified'}
- Storage: {query.split('has')[2].strip() if 'and has' in query else 'Not specified'}

Additional Considerations:
1. Please verify the exact specifications with Dell's website
2. Consider warranty and support options
3. Check for any current promotions or discounts

Would you like more specific information about any of these aspects?"""
        return {'response': formatted_response, 'status': 'success'}
    else:
        return {'response': "I apologize, but I couldn't generate a valid recommendation. Please try rephrasing your request.", 'status': 'error'}

# Create the chain with proper response handling
chain = (
    RunnablePassthrough.assign(
        context=lambda x: log_retriever_output(x["question"])
    )
    | log_prompt_output
    | log_llm_output
    | StrOutputParser()
)
logger.info("Created recommendation chain")

def recommend_laptop(state: Dict[str, Any]) -> Dict[str, Any]:
    query = state['query']
    logger.info(f"Processing recommendation request for query: {query}")
    
    try:
        logger.info("Invoking recommendation chain")
        logger.info(f"Input to chain: {{'question': {query}}}")
        response = chain.invoke({"question": query})
        logger.info(f"Chain response: {response}")
        
        result = process_response(response, query)
        logger.info(f"Processed result: {result}")
        return result
    except Exception as e:
        logger.error(f"Error in recommendation process: {str(e)}", exc_info=True)
        return {'response': f"An error occurred while processing your request: {str(e)}", 'status': 'error'}

# Update the graph
graph = Graph()
graph.add_node('recommendation', recommend_laptop)
graph.set_entry_point('recommendation')
app = graph.compile()
logger.info("Graph compiled successfully")

# Test the recommendation system
def test_recommendation(query: str) -> None:
    logger.info(f"Testing with query: {query}")
    try:
        logger.info("Invoking graph with query")
        result = app.invoke({'query': query})
        logger.info(f"Raw result from graph: {result}")
        
        if result and isinstance(result, dict) and 'response' in result:
            print('Response:', result['response'])
            
            # Analysis of the response
            print('\nAnalysis:')
            print('1. Did the agent identify the specific processor requirement?')
            print('2. Did it consider portability/travel aspects?')
            print('3. Did it provide specific model recommendations?')
            print('4. Did it mention any additional relevant features for travel use?')
        else:
            print("Error: Invalid response format received from the agent")
            logger.error(f"Invalid result format: {result}")
    except Exception as e:
        print(f"Error during recommendation: {str(e)}")
        logger.error(f"Error during recommendation: {str(e)}", exc_info=True)

# Test cases
print("\n=== Test Case 1 ===")
query1 = "I want a dell computer for travel that has Intel® Core™ 7 150U."
test_recommendation(query1)

print("\n=== Test Case 2 ===")
query2 = "I want a dell computer that has Intel® Core™ Ultra 5 135U vPro® and has 512 GB SSD."
test_recommendation(query2)

print("\n=== Test Case 3 ===")
query3 = "I want a dell computer that has Intel® Core™ Ultra 7 165U vPro® and 1 TB SSD."
test_recommendation(query3)

print("\n=== Test Case 4 ===")
query4 = "I want a light weight XPS computer with Intel® Core™ Ultra 7 165U vPro® and 1 TB SSD."
test_recommendation(query4)

INFO:__main__:Created recommendation chain
INFO:__main__:Graph compiled successfully
INFO:__main__:Testing with query: I want a dell computer for travel that has Intel® Core™ 7 150U.
INFO:__main__:Invoking graph with query
INFO:__main__:Processing recommendation request for query: I want a dell computer for travel that has Intel® Core™ 7 150U.
INFO:__main__:Invoking recommendation chain
INFO:__main__:Input to chain: {'question': I want a dell computer for travel that has Intel® Core™ 7 150U.}
INFO:__main__:Retriever received query: I want a dell computer for travel that has Intel® Core™ 7 150U.



=== Test Case 1 ===


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:__main__:Retriever returned 3 documents
INFO:__main__:Document 1 content: SpecsFeaturesReviewsSupportTech SpecsProcessorIntel® Core™ Ultra 7 155H (16 cores, up to 4.8 GHz)Operating System(Dell Technologies recommends Windows 11 Pro for business)Windows 11 Home, English, Fre...
INFO:__main__:Document 2 content: SpecsFeaturesReviewsSupportTech SpecsProcessorIntel® Core™ Ultra 7 155H (16 cores, up to 4.8 GHz)Operating System(Dell Technologies recommends Windows 11 Pro for business)Windows 11 Home, English, Fre...
INFO:__main__:Document 3 content: SpecsFeaturesReviewsSupportTech SpecsProcessorIntel® Core™ Ultra 7 155H (16 cores, up to 4.8 GHz)Operating System(Dell Technologies recommends Windows 11 Pro for business)Windows 11 Home, English, Fre...
INFO:__main__:Combined context length: 2987 characters
INFO:__main__:Formatting prompt with context and question
INFO:__main__:Input to prompt formatting: {

Error: Invalid response format received from the agent

=== Test Case 2 ===


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:__main__:Retriever returned 3 documents
INFO:__main__:Document 1 content: vPro®Windows 11 ProIntel® Graphics16 GB DDR5512 GB SSD14" Touch FHD Tech Specs  Select Selected Configuration 3   Custom Order  Starting at $1,249.00 Choose from all available specs to build a custom ...
INFO:__main__:Document 2 content: vPro®Windows 11 ProIntel® Graphics16 GB DDR5512 GB SSD14" Touch FHD Tech Specs  Select Selected Configuration 3   Custom Order  Starting at $1,249.00 Choose from all available specs to build a custom ...
INFO:__main__:Document 3 content: vPro®Windows 11 ProIntel® Graphics16 GB DDR5512 GB SSD14" Touch FHD Tech Specs  Select Selected Configuration 3   Custom Order  Starting at $1,249.00 Choose from all available specs to build a custom ...
INFO:__main__:Combined context length: 2999 characters
INFO:__main__:Formatting prompt with context and question
INFO:__main__:Input to prompt formatting: {

Error: Invalid response format received from the agent

=== Test Case 3 ===


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:__main__:Retriever returned 3 documents
INFO:__main__:Document 1 content: SpecsFeaturesReviewsSupportTech SpecsProcessorIntel® Core™ Ultra 7 155H (16 cores, up to 4.8 GHz)Operating System(Dell Technologies recommends Windows 11 Pro for business)Windows 11 Home, English, Fre...
INFO:__main__:Document 2 content: SpecsFeaturesReviewsSupportTech SpecsProcessorIntel® Core™ Ultra 7 155H (16 cores, up to 4.8 GHz)Operating System(Dell Technologies recommends Windows 11 Pro for business)Windows 11 Home, English, Fre...
INFO:__main__:Document 3 content: SpecsFeaturesReviewsSupportTech SpecsProcessorIntel® Core™ Ultra 7 155H (16 cores, up to 4.8 GHz)Operating System(Dell Technologies recommends Windows 11 Pro for business)Windows 11 Home, English, Fre...
INFO:__main__:Combined context length: 2987 characters
INFO:__main__:Formatting prompt with context and question
INFO:__main__:Input to prompt formatting: {

Error: Invalid response format received from the agent

=== Test Case 4 ===


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:__main__:Retriever returned 3 documents
INFO:__main__:Document 1 content: SpecsFeaturesReviewsSupportTech SpecsProcessorIntel® Core™ Ultra 7 155H (16 cores, up to 4.8 GHz)Operating System(Dell Technologies recommends Windows 11 Pro for business)Windows 11 Home, English, Fre...
INFO:__main__:Document 2 content: SpecsFeaturesReviewsSupportTech SpecsProcessorIntel® Core™ Ultra 7 155H (16 cores, up to 4.8 GHz)Operating System(Dell Technologies recommends Windows 11 Pro for business)Windows 11 Home, English, Fre...
INFO:__main__:Document 3 content: SpecsFeaturesReviewsSupportTech SpecsProcessorIntel® Core™ Ultra 7 155H (16 cores, up to 4.8 GHz)Operating System(Dell Technologies recommends Windows 11 Pro for business)Windows 11 Home, English, Fre...
INFO:__main__:Combined context length: 2987 characters
INFO:__main__:Formatting prompt with context and question
INFO:__main__:Input to prompt formatting: {

Error: Invalid response format received from the agent


## Summary of Agent Performance

1. Consistency in Response Format
- The agent consistently provides structured responses with clear sections
- Maintains a professional tone throughout all recommendations
- Limitations
  - Some responses are cut off mid-sentence (e.g., Test Case 4)

2. Accuracy of Recommendations
- Correctly identifies processor specifications
- Accurately matches storage requirements
- Limitations
  - Sometimes suggests upgrades when exact matches aren't available
  - Inconsistent in mentioning alternative options

3. Completeness of Information
- Includes key features and specifications
- Provides limitations and considerations
- Limitations
  - Missing pricing information in some responses
  - Incomplete responses in some cases

4. Coherence of Recommendations
- Maintains logical flow in explanations
- Provides relevant context for recommendations
- Limitations
  - Sometimes repeats information unnecessarily
  - Inconsistent in addressing all customer requirements

## Recommendations for Improvement

1. Standardize Response Format
- Implement fixed template with things like: Model name and price; Exact specifications; Available alternatives; Warranty information

2. Enhance Knowledge Base
- Add current model information
- Include complete pricing details
- Add available configurations
- Include warranty information

3. Improve Response Handling
- Better error handling for incomplete responses
- Clear alternative suggestions
- Consistent pricing information
- Complete support details

4. Add Validation
- Check response completeness
- Verify required information
- Ensure alternative suggestions
- Validate price information